# purpose

- **set parameters**
  - this notebook centralizes all functions and queries used in the repository
  - all package imports and setting parameters is done here, to avoid redundant actions
- **execution optional**
  - this notebook is called by all other notebooks in this repository
  - so you can optionally run this notebook, or you need not


# import packages


In [ ]:
# import required libraries
import requests  # used for http requests to github's api
import json      # handles json data manipulation
import pandas as pd  # pandas library for data analysis and manipulation
import csv  # handles csv file operations
import os  # provides a way of using operating system dependent functionality
import getpass  # portable password input
import subprocess  # subprocess management
from datetime import datetime, timedelta  # date and time handling


# graphql endpoint

In [ ]:
# define the graphql endpoint for github
graphql_endpoint = "https://api.github.com/graphql"

# note: for github enterprise, the endpoint url will differ
# refer to github documentation for enterprise-specific endpoints -https://docs.github.com/en/enterprise-server@3.8/graphql/guides/forming-calls-with-graphql#the-graphql-endpoint


# github token


- **need for a token**:
  - interact with the github graphql api using a github token
  - [learn how to create a token](https://docs.github.com/en/graphql/guides/forming-calls-with-graphql#authenticating-with-graphql)
  - **secure approach**:
  - this script securely stores your github token in a `github_token.config` file
  - the `.gitignore` file is updated to ensure this config file is never accidentally shared


In [ ]:
# function to find the git repository root
# we want to always create the github_config file at the root of the repo
# additionally, we want the gitignore at the root of the repo to be updated
def get_git_root():
    """get the root directory of the git repository"""
    try:
        git_root = subprocess.check_output(['git', 'rev-parse', '--show-toplevel'], universal_newlines=True).strip()
        return git_root
    except subprocess.CalledProcessError:
        raise Exception("failed to find the root of the git repository. make sure you are inside a git repository")

# get the root path of the git repository
# calls the function above
repo_root = get_git_root()

# set paths to the config file and .gitignore at the root of the repository
# uses the output of the previous function
config_filename = 'github_token.config'
config_filepath = os.path.join(repo_root, config_filename)
gitignore_filepath = os.path.join(repo_root, '.gitignore')

# first check if GITHUB_TOKEN already exists in the config file
# if yes, then skip adding it; else run it
# function to check if the config file has a valid GITHUB_TOKEN entry
def token_exists():
    if os.path.exists(config_filepath):
        with open(config_filepath, 'r') as file:
            for line in file:
                if line.startswith('GITHUB_TOKEN'):
                    # check if a token value is present after 'GITHUB_TOKEN = '
                    token_value = line.strip().split('=')[-1].strip().strip("'\"")
                    return bool(token_value)  # returns True if there's a token
    return False

# some malicious actor might remove github_token.config from .gitignore
# and then you end up uploading your token to github
# so this function checks if github_token.config is in .gitignore, and adds it if not
def ensure_gitignore_entry():
    """ensure the config file is listed in .gitignore"""
    if os.path.exists(gitignore_filepath):
        with open(gitignore_filepath, 'r') as file:
            lines = file.readlines()
        if config_filename + '\n' in lines or config_filename in lines:
            print(f"{config_filename} is already in .gitignore")
            return

    with open(gitignore_filepath, 'a') as file:
        file.write(f'{config_filename}\n')
    print(f"added '{config_filename}' to .gitignore")

# ensure that the config file is listed in .gitignore
ensure_gitignore_entry()

# check if a valid token is already present in the config file
if token_exists():
    print("GITHUB_TOKEN already exists in the config file. skipping token creation")
else:
    # get token with input masking
    token = getpass.getpass("please enter your GitHub token: ")

    # prepend the string 'token '
    token = "token " + token

    # create or overwrite the config file
    with open(config_filepath, 'w') as file:
        file.write("GITHUB_TOKEN = '{}'\n".format(token))

    print(f"config file '{config_filepath}' created successfully.\n")

# assuming this is not the first run, the existing token is picked up from the config file
def load_github_token():
    """load the GitHub token from the config file"""
    with open(config_filepath) as file:
        for line in file:
            if line.startswith('GITHUB_TOKEN'):
                # extract the token from the line
                token = line.split('=')[1].strip().strip("'")
                return token

    # raise an error if the token was not found
    raise ValueError(f"Could not find GITHUB_TOKEN in '{config_filepath}'")

# store the token in a variable
github_token = load_github_token()

# set headers for query with the loaded token
headers = {"Authorization": github_token}


# graphql queries


- **Execute a GraphQL query**
  - using the provided query string and variables.
- **Parameters**
  - `query (str)`: A GraphQL query string that will be sent to the server.
  - `variables (dict)`: A dictionary containing the variables for the query.
- **Returns**
  - `dict`: The JSON response from the GraphQL server if the request is successful.
- **Raises**
  - `Exception`: If the HTTP request fails or the server returns a non-200 status code.
- **Example usage**
  - `result = global_query("{ viewer { login } }", {})`


In [ ]:
# execute query

def global_query(query, variables):
    request = requests.post(graphql_endpoint, json={
                            'query': query, 'variables': variables}, headers=headers)
    if request.status_code == 200:
        return request.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(
            request.status_code, query))


In [ ]:
# check api quota usage
# this query is used in this file - ./100-set_parameters/120-check_usage.ipynb

check_api_quota_usage = """
{
  viewer {
    login
  }
  rateLimit {
    limit
    cost
    remaining
    resetAt
  }
}
"""
check_usage_empty_variable = {
}


In [ ]:
# fetch recently opened issues

fetch_recent_issues = """
query ($number_of_issues: Int!, $repository_name : String!, $owner_name:String!) {
  viewer {
    login
  }
  repository(name: $repository_name, owner: $owner_name) {
    issues(last: $number_of_issues) {
      edges {
        node {
          id
          title
          number
          createdAt
          closedAt
          state
          updatedAt
          comments(first: 10) {
            edges {
              node {
                id
                body
              }
            }
          }
          labels(orderBy: {field: NAME, direction: ASC}, first: 10) {
            edges {
              node {
                name
              }
            }
          }
          comments(first: 10) {
            edges {
              node {
                id
                body
              }
            }
          }
        }
        cursor
      }
    }
  }
}
"""


In [ ]:
# fetch first 100 closed issues

fetch_first_100_closed_issues = """
query ($number_of_issues: Int!, $repository_name : String!, $owner_name:String!) {
  viewer {
    login
  }
  repository(name: $repository_name, owner: $owner_name) {
    issues(
      first: $number_of_issues
      orderBy: {field: CREATED_AT, direction: ASC}
    ) {
      edges {
        node {
          id
          title
          number
          createdAt
          closedAt
          state
          updatedAt
          comments(first: 100) {
            edges {
              node {
                id
                body
              }
            }
          }
          labels(orderBy: {field: NAME, direction: ASC}, first: 10) {
            edges {
              node {
                name
              }
            }
          }
          comments(first: 100) {
            edges {
              node {
                id
                body
              }
            }
          }
        }
        cursor
      }
      pageInfo {
        endCursor
        hasNextPage
        hasPreviousPage
        startCursor
      }
    }
  }
}
"""


In [ ]:
# fetch next 100 closed issues

fetch_next_100_closed_issues = """
query ($end_cursor_value: String!, $number_of_issues: Int!, $repository_name : String!, $owner_name:String!) {
  viewer {
    login
  }
  repository(name: $repository_name, owner: $owner_name) {
    issues(
      first: $number_of_issues
      orderBy: {field: CREATED_AT, direction: ASC}
      after: $end_cursor_value
    ) {
      edges {
        node {
          id
          title
          number
          createdAt
          closedAt
          state
          updatedAt
          comments(first: 100) {
            edges {
              node {
                id
                body
              }
            }
          }
          labels(orderBy: {field: NAME, direction: ASC}, first: 10) {
            edges {
              node {
                name
              }
            }
          }
          comments(first: 100) {
            edges {
              node {
                id
                body
              }
            }
          }
        }
        cursor
      }
      pageInfo {
        endCursor
        hasNextPage
        hasPreviousPage
        startCursor
      }
    }
  }
}
"""


In [ ]:
# fetch issue labels

fetch_100_labels = """
query ($repository_name : String!, $owner_name:String!) {
  repository(owner: $owner_name, name: $repository_name) {
    labels(first: 100) {
      nodes {
        id
        name
      }
    }
  }
}
"""

In [ ]:
# fetch list of repositories

fetch_100_repositories = """
query ($login_user: String!) {
  user(login: $login_user) {
    repositories(first: 100) {
      edges {
        node {
          id
          name
        }
      }
    }
  }
}
"""

In [ ]:
# fetch list of user ids

fetch_100_user_ids = """
query ($owner_name: String!,$repository_name: String!) {
  repository(owner: $owner_name, name: $repository_name) {
    collaborators(first: 100) {
      edges {
        node {
          id
          name
        }
      }
    }
  }
}
"""

In [ ]:
%%capture
# magic command to suppress output of this cell

# queries
%store check_api_quota_usage
%store fetch_100_labels
%store fetch_100_repositories
%store fetch_100_user_ids
%store fetch_first_100_closed_issues
%store fetch_next_100_closed_issues

# variables
%store check_usage_empty_variable